# Predicting if a pokemon is legendary!

## Hypothesis
##### Legendary pokemons tend to have better stats so we could predict if a pokemon is legendary or normal just from its stats

## Loading raw data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
raw_df = pd.read_csv("pokemons.csv")
raw_df.head(1)


,id,name,rank,generation,evolves_from,type1,type2,hp,atk,def,spatk,spdef,speed,total,height,weight,abilities,desc
0,1,bulbasaur,ordinary,generation-i,nothing,grass,poison,45,49,49,65,65,45,318,7,69,overgrow chlorophyll,A strange seed was planted on its back at birt...


## Extracting features and outputs

In [19]:
clean_df = pd.DataFrame()
clean_df = raw_df[["hp", "atk", "def", "spatk", "spdef", "speed"]]
y  = raw_df["rank"].apply(lambda x: 1 if x == "legendary" else 0)
